In [1]:
RANDOM_STATE = 1

In [2]:
#!pip install spacy

In [3]:
#!python -m spacy download en

# Imports

In [2]:
import pandas as pd
import time
import numpy as np
import pickle
from sklearn.metrics import confusion_matrix
import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_colwidth',300)

from nltk.corpus import stopwords
import re
import nltk
from nltk import tokenize
from nltk.stem import WordNetLemmatizer

import math

import spacy
import os
#import pyprojroot.here as here

# Import datasets from drive

Note - for the purpose of brevity, data has already been preprocessed and cleaned.

In [5]:
#from google.colab import drive
#drive.mount('/content/drive')

In [3]:
RAW_FOLDER = "data\\cleaned"

In [6]:
csvs = os.listdir(r"C:\Nga\BT4222\sentiment_analysis_bt4222\data\cleaned")

In [7]:
csvs

['cleaned_crowne-plaza.csv',
 'cleaned_fullerton.csv',
 'cleaned_grand-mercure-sg-roxy.csv',
 'cleaned_holiday-inn-express-clarke-quay.csv',
 'cleaned_hotel-boss.csv',
 'cleaned_hotel-G.csv',
 'cleaned_ibis-sg-bencoolen.csv',
 'cleaned_mbs_total.csv',
 'cleaned_pan-pacific.csv',
 'cleaned_paradox-sg-merchant-court.csv',
 'cleaned_park-regis.csv',
 'cleaned_parkroyal-collection-marina-bay.csv',
 'cleaned_swissotel-the-stamford.csv',
 'cleaned_village-hotel-albert-court-by-far-east-hospitality.csv',
 'cleaned_village-hotel-changi-by-far-east-hospitality.csv']

In [10]:
#data_path = '/content/drive/MyDrive/BT4222/data/raw/mbs_total.csv'
#df = pd.read_csv(data_path)
#df.head(10)

In [66]:
# TODO: modify these list if needed (eg. if you want to load only 1 csv from star3, delete other csvs in star3 list)
star3 = ['cleaned_ibis-sg-bencoolen.csv','cleaned_hotel-boss.csv','cleaned_hotel-G.csv',
           'cleaned_village-hotel-albert-court-by-far-east-hospitality.csv',
           'cleaned_holiday-inn-express-clarke-quay.csv']
star4 = ['cleaned_village-hotel-changi-by-far-east-hospitality.csv',
         'cleaned_park-regis.csv', 'cleaned_grand-mercure-sg-roxy.csv',
         'cleaned_paradox-sg-merchant-court.csv','cleaned_crowne-plaza.csv']
star5 = ['cleaned_fullerton.csv', 'cleaned_parkroyal-collection-marina-bay.csv', 'cleaned_pan-pacific.csv',
          'cleaned_mbs_total.csv', 'cleaned_swissotel-the-stamford.csv']
ROOT = ''
RAW_FOLDER = "..\\data\\cleaned"
def filter_date(row):
  try:
    #print(int(row["date_of_stay"][-4:]))
    return int(row["date_of_stay"][-4:])
  except:
    #print("2000")
    return 2000
def combine_csv_to_dataframe(file_names, all_star = False, filterDate = True):
    """
    Combine multiple CSV files into a single DataFrame.

    Parameters:
    file_names (list): List of CSV file names. 
    all_star (bool): whether or not to load all the hotels (False if only want to load 1 type of hotel star). 
    filterData (bool): whether or not to remove all data dated before 2015

    Returns:
    pd.DataFrame: Combined DataFrame.
    """
    combined_df = pd.DataFrame()

    for file_name in file_names:
        file_name = RAW_FOLDER + "\\" + file_name
        try:
            df = pd.read_csv(file_name)
            if all_star:
              if file_name in star3:
                df["star"] = 3
              elif file_name in star4:
                df["star"] = 4
              else:
                df["star"] = 5
            print(f"Length of {file_name} is {len(df)}")
            combined_df = pd.concat([combined_df, df], ignore_index=True)
        except FileNotFoundError:
            print(f"File not found: {file_name}")
        except pd.errors.EmptyDataError:
            print(f"Empty or invalid CSV file: {file_name}")
    combined_df["year"] = combined_df.apply(lambda row: filter_date(row), axis = 1)
    if filterDate:
      combined_df = combined_df[combined_df["year"]>=2015]
    return combined_df

In [23]:
data = combine_csv_to_dataframe(star3+star4+star5, all_star = True, filterDate = True)
#data[['traveller_username','date','travel_type','traveller_total_contributions','traveller_total_helpful_contributions','review_title','review_text','rating']].head(5)
data.info()
data.head(3)

Length of ..\data\cleaned\cleaned_ibis-sg-bencoolen.csv is 5933
Length of ..\data\cleaned\cleaned_hotel-boss.csv is 3568
Length of ..\data\cleaned\cleaned_hotel-G.csv is 3829
Length of ..\data\cleaned\cleaned_village-hotel-albert-court-by-far-east-hospitality.csv is 3285
Length of ..\data\cleaned\cleaned_holiday-inn-express-clarke-quay.csv is 2551
Length of ..\data\cleaned\cleaned_village-hotel-changi-by-far-east-hospitality.csv is 5621
Length of ..\data\cleaned\cleaned_park-regis.csv is 5298
Length of ..\data\cleaned\cleaned_grand-mercure-sg-roxy.csv is 5015
Length of ..\data\cleaned\cleaned_paradox-sg-merchant-court.csv is 5813
Length of ..\data\cleaned\cleaned_crowne-plaza.csv is 5743
Length of ..\data\cleaned\cleaned_fullerton.csv is 7975
Length of ..\data\cleaned\cleaned_parkroyal-collection-marina-bay.csv is 8129
Length of ..\data\cleaned\cleaned_pan-pacific.csv is 10797
Length of ..\data\cleaned\cleaned_mbs_total.csv is 19371
Length of ..\data\cleaned\cleaned_swissotel-the-stamf

,Unnamed: 0,date_of_stay,traveller_username,review_title,review_text,travel_type,traveller_country_origin,traveller_total_contributions,traveller_total_helpful_contributions,rating1,rating2,rating,valid_rating,label,cleaned_review,combined_review,date,covid,star,year
0,0,Date of stay: August 2023,Love_Life_Sydney,Clean and comfortable,"Hotel rooms in Singapore are so expensive so to find a decent hotel, with easy transport access and food locations, at less than S$200 was a good deal. The compact room meets your basic needs, no fancy mood lightning or lovely artworks to grace the wall. It has clean sheets and towels, a small b...",Trip type: Travelled as a couple,"Sydney, Australia",2302.0,871.0,NaN,4.0,4.0,True,Positive,clean comfortable hotel room singapore expensive find decent hotel easy transport access food location less good deal compact room meet basic need fancy mood lightning lovely artwork grace wall clean sheet towel small bathroom pump pack amenity kettle glass bottle refill small selection tea coff...,"Clean and comfortable Hotel rooms in Singapore are so expensive so to find a decent hotel, with easy transport access and food locations, at less than S$200 was a good deal. The compact room meets your basic needs, no fancy mood lightning or lovely artworks to grace the wall. It has clean sheets...",2023-08-01,PostCovid,5,2023
1,1,Date of stay: August 2023,Bilal S,"Good hotel, great location",This is a great place! Location is great but the room is very small. There is no room service available. Overall it's ok and they were kind enough to hold my luggage after check out! It is a good hotel!!,Trip type: Travelled with family,"Houston, Texas",4.0,NaN,5.0,NaN,5.0,True,Positive,good hotel great location great place location great room small room service available overall ok kind enough hold luggage check good hotel,"Good hotel, great location This is a great place! Location is great but the room is very small. There is no room service available. Overall it's ok and they were kind enough to hold my luggage after check out! It is a good hotel!!",2023-08-01,PostCovid,5,2023
2,2,Date of stay: October 2022,Anthony Fernando,Good place for a decent price.,Good place good price Easy access to the city. All walking distance. Very close to the buggies junction. Food comers around you. Also walking distance to marina bay sands Cozy rooms. Easy checking and check out. Worth for the price. Quick getaway.,Trip type: Travelled with friends,"Dubai, United Arab Emirates",39.0,38.0,5.0,NaN,5.0,True,Positive,good place decent price good place good price easy access city walk distance close buggy junction food comer around also walk distance marina bay sand cozy room easy check check worth price quick getaway,Good place for a decent price. Good place good price Easy access to the city. All walking distance. Very close to the buggies junction. Food comers around you. Also walking distance to marina bay sands Cozy rooms. Easy checking and check out. Worth for the price. Quick getaway.,2022-10-01,PostCovid,5,2022


In [33]:
data.columns

Index(['Unnamed: 0', 'date_of_stay', 'traveller_username', 'review_title',
       'review_text', 'travel_type', 'traveller_country_origin',
       'traveller_total_contributions',
       'traveller_total_helpful_contributions', 'rating1', 'rating2', 'rating',
       'valid_rating', 'label', 'cleaned_review', 'combined_review', 'date',
       'covid', 'star', 'year'],
      dtype='object')

# Clean Data

## General text preprocess (remove nonalphanum, lowercase, remove stop words)

In [27]:
# preprocess function
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
spacy_lemmatizer = spacy.load('en_core_web_sm', disable=['parser','ner'])

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ngalh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [67]:
def preprocess(text):
    # lower text
    text = text.lower()
    # Remove newline characters
    text = text.replace('\\n',' ').replace('\n', ' ').replace('\t', ' ').replace('\r', ' ').replace('\\', ' ')
    # Remove punctuation and numbers
    text = re.sub('[^a-zA-Z]', ' ', text)
    # Remove multiple spaces
    text = re.sub(r'\s+',' ', text)
    # remove stop word
    text = text.split()
    text = ' '.join([word for word in text if word not in stop_words])
    return text

## Lemmatize and Stemming

In [45]:
from nltk.stem.porter import *
from nltk.stem.snowball import SnowballStemmer

In [68]:
def lem_stem(text, lem = False, stem = None):
    text = preprocess(text)
    # lemmatize
    if lem:
        text = spacy_lemmatizer(text)
        text = ' '.join([token.lemma_ for token in text])
    # Remove stop words
    if stem:
        if stem == "porter":
            stemmer = PorterStemmer()
        else: 
            stemmer = SnowballStemmer("english")
        text = ' '.join([stemmer.stem(word) for word in text.split()])
    return text

In [49]:
data["stem_review"] = data.apply(lambda row: lem_stem(row['combined_review'], lem = False, stem = "porter"), axis = 1)

In [54]:
data["lem_review"] = data.apply(lambda row: lem_stem(row['combined_review'], lem = True, stem = None), axis = 1)


In [57]:
data["cleaned_review"] = data.apply(lambda row: preprocess(row['combined_review']), axis = 1)


In [59]:
remove_col = ['rating1', 'rating2', 'Unnamed: 0', 'date_of_stay', 'valida_rating']

In [61]:
data = data.drop(columns = remove_col)

In [70]:
for csv in star3+star4+star5:
    data = combine_csv_to_dataframe([csv], all_star=False, filterDate=True)
    data["stem_review"] = data.apply(lambda row: lem_stem(row['combined_review'], lem = False, stem = "porter"), axis = 1)
    data["lem_review"] = data.apply(lambda row: lem_stem(row['combined_review'], lem = True, stem = None), axis = 1)
    data["cleaned_review"] = data.apply(lambda row: preprocess(row['combined_review']), axis = 1)
    data = data.drop(columns = remove_col)
    data.to_csv(f"../data/processed/{csv}")
    


Length of ..\data\cleaned\cleaned_ibis-sg-bencoolen.csv is 5933
Length of ..\data\cleaned\cleaned_hotel-boss.csv is 3568
Length of ..\data\cleaned\cleaned_hotel-G.csv is 3829
Length of ..\data\cleaned\cleaned_village-hotel-albert-court-by-far-east-hospitality.csv is 3285
Length of ..\data\cleaned\cleaned_holiday-inn-express-clarke-quay.csv is 2551
Length of ..\data\cleaned\cleaned_village-hotel-changi-by-far-east-hospitality.csv is 5621
Length of ..\data\cleaned\cleaned_park-regis.csv is 5298
Length of ..\data\cleaned\cleaned_grand-mercure-sg-roxy.csv is 5015
Length of ..\data\cleaned\cleaned_paradox-sg-merchant-court.csv is 5813
Length of ..\data\cleaned\cleaned_crowne-plaza.csv is 5743
Length of ..\data\cleaned\cleaned_fullerton.csv is 7975
Length of ..\data\cleaned\cleaned_parkroyal-collection-marina-bay.csv is 8129
Length of ..\data\cleaned\cleaned_pan-pacific.csv is 10797
Length of ..\data\cleaned\cleaned_mbs_total.csv is 19371
Length of ..\data\cleaned\cleaned_swissotel-the-stamf

In [64]:
data.to_csv("../data/processed/test.csv")

KeyboardInterrupt: 

In [62]:
data

,traveller_username,review_title,review_text,travel_type,traveller_country_origin,traveller_total_contributions,traveller_total_helpful_contributions,rating,valid_rating,label,cleaned_review,combined_review,date,covid,star,year,stem_review,lem_review
0,Love_Life_Sydney,Clean and comfortable,"Hotel rooms in Singapore are so expensive so to find a decent hotel, with easy transport access and food locations, at less than S$200 was a good deal. The compact room meets your basic needs, no fancy mood lightning or lovely artworks to grace the wall. It has clean sheets and towels, a small b...",Trip type: Travelled as a couple,"Sydney, Australia",2302.0,871.0,4.0,True,Positive,clean comfortable hotel rooms singapore expensive find decent hotel easy transport access food locations less good deal compact room meets basic needs fancy mood lightning lovely artworks grace wall clean sheets towels small bathroom pump pack amenities kettle glass bottle refills small selectio...,"Clean and comfortable Hotel rooms in Singapore are so expensive so to find a decent hotel, with easy transport access and food locations, at less than S$200 was a good deal. The compact room meets your basic needs, no fancy mood lightning or lovely artworks to grace the wall. It has clean sheets...",2023-08-01,PostCovid,5,2023,clean comfort hotel room singapor expens find decent hotel easi transport access food locat less good deal compact room meet basic need fanci mood lightn love artwork grace wall clean sheet towel small bathroom pump pack amen kettl glass bottl refil small select tea coffe small fridg room safe c...,clean comfortable hotel room singapore expensive find decent hotel easy transport access food location less good deal compact room meet basic need fancy mood lightne lovely artwork grace wall clean sheet towel small bathroom pump pack amenity kettle glass bottle refill small selection tea coffee...
1,Bilal S,"Good hotel, great location",This is a great place! Location is great but the room is very small. There is no room service available. Overall it's ok and they were kind enough to hold my luggage after check out! It is a good hotel!!,Trip type: Travelled with family,"Houston, Texas",4.0,NaN,5.0,True,Positive,good hotel great location great place location great room small room service available overall ok kind enough hold luggage check good hotel,"Good hotel, great location This is a great place! Location is great but the room is very small. There is no room service available. Overall it's ok and they were kind enough to hold my luggage after check out! It is a good hotel!!",2023-08-01,PostCovid,5,2023,good hotel great locat great place locat great room small room servic avail overal ok kind enough hold luggag check good hotel,good hotel great location great place location great room small room service available overall ok kind enough hold luggage check good hotel
2,Anthony Fernando,Good place for a decent price.,Good place good price Easy access to the city. All walking distance. Very close to the buggies junction. Food comers around you. Also walking distance to marina bay sands Cozy rooms. Easy checking and check out. Worth for the price. Quick getaway.,Trip type: Travelled with friends,"Dubai, United Arab Emirates",39.0,38.0,5.0,True,Positive,good place decent price good place good price easy access city walking distance close buggies junction food comers around also walking distance marina bay sands cozy rooms easy checking check worth price quick getaway,Good place for a decent price. Good place good price Easy access to the city. All walking distance. Very close to the buggies junction. Food comers around you. Also walking distance to marina bay sands Cozy rooms. Easy checking and check out. Worth for the price. Quick getaway.,2022-10-01,PostCovid,5,2022,good place decent price good place good price easi access citi walk distanc close buggi junction food comer around also walk distanc marina bay sand cozi room easi check check worth price qu